In [2]:
## Import libraries
import googletrans
import pandas as pd

## Load and Explore  Labels Data

In [1]:
#Import labeled data downloaded from : http://gapfinder-tools.wmflabs.org/admin/sectionalignment/userinput/
#A copy of can be downlodaded from: ADD GITHUB LINK HERE
import pandas as pd

labels = pd.read_csv('SectionTranslationLabels.tsv',sep='\t')
labels['destination_title'] = labels.destination_title.map(eval) #transform string to list

In [2]:
total = 0
for src,data in labels.groupby(labels.source_lang):
    for dest,data2 in data.groupby('destination_language'):
        print(src,dest,len(data2))
        total+=len(data2)

ar en 544
ar es 87
ar fr 276
ar ja 11
ar ru 27
en ar 473
en es 999
en fr 999
en ja 698
en ru 947
es ar 3
es en 526
es fr 500
es ja 1
es ru 19
fr ar 42
fr en 529
fr es 142
fr ja 76
fr ru 52
ja en 100
ja es 10
ja fr 77
ru ar 3
ru en 418
ru es 18
ru fr 332
ru ja 37


In [4]:
### Exploid multiple translatios
df = labels.set_index(['source_lang', 'source_title','destination_language'])['destination_title'].apply(pd.Series).stack()
df = df.reset_index()
del(df['level_3'])
df.rename(columns={0:'destination_title'},inplace=True)
df.set_index('source_lang',inplace=True)
labelsExploid = df.reset_index() 

In [6]:
total = 0
for src,data in df.groupby(df.index):
    for dest,data2 in data.groupby('destination_language'):
        print(src,dest,len(data2))
        total+=len(data2)
print(total)

ar en 565
ar es 87
ar fr 285
ar ja 21
ar ru 29
en ar 543
en es 1062
en fr 1210
en ja 1169
en ru 1178
es ar 3
es en 569
es fr 522
es ja 2
es ru 19
fr ar 43
fr en 746
fr es 155
fr ja 162
fr ru 64
ja en 129
ja es 12
ja fr 105
ru ar 3
ru en 460
ru es 19
ru fr 346
ru ja 41
9549


In [10]:
langs = list(df.index.unique())

### Translations

Get translations can take around 1 hour, **run this cell just if you want to refresh translations**, if not just use the saved versions in 'translations.p'

In [8]:
#COMMENT THIS LINE TO RUN THIS CELL 

from time import sleep 
from googletrans import Translator
import random
import requests
import pandas as pd

#### Proxies for use googletrans #####
def getFreeProxies(transparent=True): # get list of proxies from https://free-proxy-list.net/ 
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    tmp = pd.read_html(response.text)[0]
    output = {'http':[],'https':[]}
    tmp.dropna(inplace=True)
    if transparent:
        tmp = tmp[tmp.Anonymity == 'transparent'] #using just transparent proxies, 
    for i,proxy in tmp.iterrows():
        if proxy['Https'] == 'no':
            output['http'].append(proxy['IP Address']+':'+str(int(proxy['Port'])))
        else:
            output['https'].append(str(proxy['IP Address'])+':'+str(int(proxy['Port'])))
    return output
proxies=getFreeProxies()['http']

 

translations = pd.DataFrame([]) # Init translations results

for lang in langs:
    print(lang)
    for lang2 in set(langs):
        srcs = list(df[(df.index ==lang) & (df.destination_language ==lang2)]['source_title'].unique())
        if lang != lang2:
            print('---',lang2)
            translated = False
            while not translated:
                try:
                     #I use uk translator, to have a fixed engine
                    # and iterate over proxies until get translation
                    translator = Translator(service_urls=['translate.google.co.uk'],
                                    proxies={'http':random.choice(proxies)},timeout=15)
                    result = translator.translate(srcs,src=lang,dest=lang2)
                    translated =  True
                except:
                    pass
            pairs = [(lang,sec.origin,lang2,sec.text) for sec in result]
            pairs = pd.DataFrame(pairs,columns=['source_lang','source_title','destination_language','automatic_translation'])
            translations = pd.concat([translations,pairs])
    #sleep(60)

translations.to_pickle('translations.p')

ru
--- fr
--- ja
--- en
--- es
--- ar
ja
--- fr
--- en
--- es
--- ru
--- ar
fr
--- ja
--- en
--- es
--- ru
--- ar
es
--- fr
--- ja
--- en
--- ru
--- ar
en
--- fr
--- ja
--- es
--- ru
--- ar
ar
--- fr
--- ja
--- en
--- es
--- ru


### Check Quality of Automatic Translations

In [7]:
translations =pd.read_pickle('translations.p')

In [8]:
df = pd.merge(df, translations,  how='left', left_on=[df.index,'source_title','destination_language'], right_on = ['source_lang','source_title','destination_language'],suffixes=('', '_dup'))

In [13]:
automaticTranslationQuality = []
for lang1,data in df.groupby('source_lang'):
    for lang2,data2 in data.groupby('destination_language'):
        tmp = (data2.destination_title.str.lower()== data2.automatic_translation.str.lower())
        automaticTranslationQuality.append((lang1,lang2,dict(tmp.value_counts()).get(True,0),dict(tmp.value_counts())[False]))
automaticTranslationQuality = pd.DataFrame(automaticTranslationQuality,columns=['source_lang','dest_lang','Correct','Incorrect'])
automaticTranslationQuality['accuracy'] = automaticTranslationQuality.Correct / (automaticTranslationQuality.Correct + automaticTranslationQuality.Incorrect)


In [509]:
automaticTranslationQuality.sort_values('accuracy',ascending=False)

,source_lang,dest_lang,Correct,Incorrect,accuracy
14,es,ru,14,5,0.736842
17,fr,es,104,51,0.670968
6,en,es,707,355,0.665725
4,ar,ru,19,10,0.655172
11,es,en,336,233,0.590510
1,ar,es,50,37,0.574713
25,ru,es,10,9,0.526316
24,ru,en,242,218,0.526087
12,es,fr,261,261,0.500000
19,fr,ru,31,33,0.484375
